In [1]:
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [2]:
# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

0it [00:00, ?it/s]Training MNIST Model on cpu
 99%|█████████▊| 9781248/9912422 [00:04<00:00, 3360862.69it/s]Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz

  0%|          | 0/28881 [00:01<?, ?it/s]

0it [00:00, ?it/s]Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw


  0%|          | 0/1648877 [00:01<?, ?it/s]

  1%|          | 16384/1648877 [00:01<00:22, 72504.20it/s]

  3%|▎         | 49152/1648877 [00:01<00:10, 145860.74it/s]

  6%|▌         | 98304/1648877 [00:01<00:07, 214253.38it/s]

  9%|▉         | 155648/1648877 [00:01<00:06, 235446.36it/s]

 18%|█▊        | 303104/1648877 [00:01<00:02, 495462.07it/s]

 29%|██▉       | 475136/1648877 [00:01<00:01, 776592.86it/s]

 37%|███▋      | 606208/1648877 [00:02<00:01, 754005.80it/s]

 52%|█████▏    | 860160/1648877 [00

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

32768it [00:17, 1915.38it/s]             
1654784it [00:16, 103287.94it/s] 
8192it [00:12, 631.05it/s]              


In [4]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [6]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')

 | Batch Status: 51200/60000 (85%) | Loss: 0.130805
Train Epoch: 6 | Batch Status: 51840/60000 (86%) | Loss: 0.103187
Train Epoch: 6 | Batch Status: 52480/60000 (87%) | Loss: 0.227153
Train Epoch: 6 | Batch Status: 53120/60000 (88%) | Loss: 0.091589
Train Epoch: 6 | Batch Status: 53760/60000 (90%) | Loss: 0.156986
Train Epoch: 6 | Batch Status: 54400/60000 (91%) | Loss: 0.241512
Train Epoch: 6 | Batch Status: 55040/60000 (92%) | Loss: 0.305819
Train Epoch: 6 | Batch Status: 55680/60000 (93%) | Loss: 0.090115
Train Epoch: 6 | Batch Status: 56320/60000 (94%) | Loss: 0.094693
Train Epoch: 6 | Batch Status: 56960/60000 (95%) | Loss: 0.206353
Train Epoch: 6 | Batch Status: 57600/60000 (96%) | Loss: 0.055983
Train Epoch: 6 | Batch Status: 58240/60000 (97%) | Loss: 0.112256
Train Epoch: 6 | Batch Status: 58880/60000 (98%) | Loss: 0.054385
Train Epoch: 6 | Batch Status: 59520/60000 (99%) | Loss: 0.317952
Training time: 0m 10s
Test set: Average loss: 0.0023, Accuracy: 9568/10000 (96%)
Testing t